## Create a Feast Feature Store from Parquet Files as DataSource
![](images/feast_fs.png)

In [2]:
import sys
sys.path.insert(0, "../")

#### Import General Python libs and modules

In [3]:
from datetime import datetime
from pathlib import Path
import pandas as pd
from pprint import pprint

#### Import Feast related modules and definitions from feast_repo directories

In [4]:
from feast import FeatureStore
from entities.entity import zipcode, dob_ssn
from features.feature_views import zipcode_features, credit_history
from feature_service.feature_svc import zipcode_features_svc
from utils.data_fetcher import DataFetcher

/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/feast/data_source.py:229: DeprecationWarning: The argument 'event_timestamp_column' is being deprecated. Please use 'timestamp_field' instead. instead. Feast 0.23 and onwards will not support the argument 'event_timestamp_column' for datasources.
  warnings.warn(
/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/feast/feature_view.py:256: DeprecationWarning: batch_source and stream_source have been deprecated in favor of `source`.The deprecated fields will be removed in Feast 0.23.
  warnings.warn(
/Users/kike/opt/miniconda3/envs/feast_workshop/lib/python3.10/site-packages/feast/feature_view.py:194: DeprecationWarning: The `features` parameter is being deprecated in favor of the `schema` parameter. Please switch from using `features` to `schema`. This will also requiring switching feature definitions from using `Feature` to `Field`. Feast 0.23 and onwards will not support the `features

In [7]:
# [IMPORTANT] Change this  path to yours git repo
FEAST_REPO = "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_3/feature_repo"
repo_path = Path(FEAST_REPO)
fs = FeatureStore(repo_path=repo_path)

#### Step 1. Register the data source, entity, features and feature view, and feature service in feast_repo with the Feast Registry

In [41]:
#print(f"Zip code entity:{zipcode}\n")
#print(f"Date-of-birth and SSN entity:{dob_ssn}\n")
#print(f"ZIP code features:{zipcode_features}\n")
#print(f"Credit history features:{credit_history}\n")
#print(f"ZIP code SVC features:{zipcode_features_svc}\n")

In [8]:
fs.apply([zipcode, dob_ssn, zipcode_features, credit_history, zipcode_features_svc])

#### Step 2. Now materialize, load data into online store

In [54]:
start_date = datetime(2020, 3, 1, 0, 0, 0)
end_date = datetime(2020, 6, 1,  0, 0, 0)
#fs.materialize_incremental(end_date=end_date)
fs.materialize(start_date, end_date)

Materializing 2 feature views from 2020-03-01 00:00:00-06:00 to 2020-06-01 00:00:00-05:00 into the sqlite online store.

zipcode_features:


0it [00:00, ?it/s]

credit_history:



100%|███████████████████████████████████████████████████████| 28633/28633 [00:07<00:00, 3647.86it/s]


In [40]:
#for fview in fs.list_feature_views():
#    print(">>>> FEATURE:", fview,"\n")

### Step 3. Fetch training data augmented with loans data

In [34]:
fetcher = DataFetcher(fs, repo_path)
training_df = fetcher.get_training_data()
pd.set_option('display.max_columns', 50)
training_df.head(5)

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp__,city,state,location_type,tax_returns_filed,population,total_wages,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies
1358886,38450,19721022_2735,70460,55,24543,RENT,3.0,VENTURE,4000,13.92,0,2020-08-28 05:46:51.871000+00:00,2017-01-01 12:00:00+00:00,SLIDELL,LA,PRIMARY,8986,16334,315061217,1777,690650,46372,10439,5,1,2,1,0
1358815,38449,19550130_7970,74337,58,20000,RENT,0.0,EDUCATION,4000,9.99,0,2020-08-28 06:05:13.103000+00:00,2017-01-01 12:00:00+00:00,CHOUTEAU,OK,PRIMARY,1969,3716,59412230,1791,462670,19421,3583,8,7,1,0,2
1353348,38554,19970610_1743,58504,64,24000,RENT,1.0,MEDICAL,3000,6.99,0,2020-08-26 21:58:03.673000+00:00,2017-01-01 12:00:00+00:00,BISMARCK,ND,PRIMARY,11564,19832,469621263,5917,1780959,11835,27910,8,3,2,1,0
1354200,38552,19650412_2624,93160,55,34000,RENT,0.0,DEBTCONSOLIDATION,12000,6.92,1,2020-08-26 22:34:46.138000+00:00,2017-01-01 12:00:00+00:00,SANTA BARBARA,CA,PRIMARY,602,909,24537583,8091,364271,30248,22640,2,7,3,0,0
1354271,38541,19681130_4866,92615,51,74628,MORTGAGE,3.0,PERSONAL,3000,13.49,0,2020-08-27 01:56:39.697000+00:00,2017-01-01 12:00:00+00:00,HUNTINGTON BEACH,CA,PRIMARY,513,746,19749601,3679,1659968,37582,20284,0,1,0,0,0


In [35]:
training_df.shape

(28638, 28)

#### Step 4. Get an inference vector for zipcode and dob_ssn

In [55]:
zipcodes_dob_ssns = [(70460, "19721022_2735"), (74337, "19550130_7970")]
for zipcode, dob_ssn in zipcodes_dob_ssns:
    print(f"Fetching feature vector for zipcode and dob_ssn:{zipcode}, {dob_ssn}")
    data = fetcher.get_online_data(zipcode, dob_ssn)
    pprint(data)
    print("------" * 5)

Fetching feature vector for zipcode and dob_ssn:70460, 19721022_2735
{'bankruptcies': [0],
 'city': ['SLIDELL'],
 'credit_card_due': [1777],
 'dob_ssn': ['19721022_2735'],
 'hard_pulls': [5],
 'location_type': ['PRIMARY'],
 'missed_payments_1y': [2],
 'missed_payments_2y': [1],
 'missed_payments_6m': [1],
 'mortgage_due': [690650],
 'population': [16334],
 'state': ['LA'],
 'student_loan_due': [46372],
 'tax_returns_filed': [8986],
 'total_wages': [315061217],
 'vehicle_loan_due': [10439],
 'zipcode': [70460]}
------------------------------
Fetching feature vector for zipcode and dob_ssn:74337, 19550130_7970
{'bankruptcies': [2],
 'city': ['CHOUTEAU'],
 'credit_card_due': [1791],
 'dob_ssn': ['19550130_7970'],
 'hard_pulls': [8],
 'location_type': ['PRIMARY'],
 'missed_payments_1y': [1],
 'missed_payments_2y': [7],
 'missed_payments_6m': [0],
 'mortgage_due': [462670],
 'population': [3716],
 'state': ['OK'],
 'student_loan_due': [19421],
 'tax_returns_filed': [1969],
 'total_wages': [

In [45]:
online_vec = fs.get_online_features(
            entity_rows=[{"zipcode": 30721, "dob_ssn": '19790429_9552'}],
            features=DataFetcher.feast_features).to_df()

In [42]:
fs.list_feature_views()

[<FeatureView(name = zipcode_features, entities = ['zipcode'], stream_source = None, batch_source = {
   "type": "BATCH_FILE",
   "timestampField": "event_timestamp",
   "createdTimestampColumn": "created_timestamp",
   "fileOptions": {
     "uri": "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_3/feature_repo/data/zipcode_table.parquet"
   },
   "name": "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_3/feature_repo/data/zipcode_table.parquet"
 }, source = {
   "type": "BATCH_FILE",
   "timestampField": "event_timestamp",
   "createdTimestampColumn": "created_timestamp",
   "fileOptions": {
     "uri": "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-master/module_3/feature_repo/data/zipcode_table.parquet"
   },
   "name": "/Users/kike/Library/CloudStorage/OneDrive-VMware,Inc/OCTO/2022-H1/Taurus/Feast/feast_workshops-